In [1]:
from ttrtypes import Type,BType,PType,Pred,Possibility,RecType,Fun,ListType,\
                     TTRStringType,KPlusStringType,LazyObj,ti_apply,SingletonType,FunType,JoinType,RecTy,Ty,Re
from records import Rec
from utils import show, substitute

class SingletonType(SingletonType):
    def __init__(self,T,a):
        self.comps = Rec({'base_type':T, 'obj':a})
        self.witness_cache = []
        self.supertype_cache = []
        self.witness_conditions = [lambda x: show(x) == show(a) and T.query(x), #x == a changed to show(x) == show(a)
                                   lambda x: isinstance(a,LazyObj)\
                                             and show(x) == show(a.eval()) and T.query(x)]
    def subst(self,v,a):
        if self == v:
            return a
        else:  
            new_base_type= substitute(self.comps.base_type,v,a)  #extracted calls to substitute
            new_obj = substitute(self.comps.obj,v,a)
            return SingletonType(new_base_type,new_obj)
        
#class RecType(RecType):
#    def subst(self,v,a):
#        res = RecType()
#        for l in self.comps.__dict__.keys():
#            if self.comps.__getattribute__(l) == v:
#                res.addfield(l,a)
#            elif isinstance(self.comps.__getattribute__(l),str):
#                res.addfield(l,self.comps.__getattribute__(l))
#            else: 
#                new_value = substitute(self.comps.__getattribute__(l),v,a) #extracted call to substitute
#                res.addfield(l,new_value)
#        #print(show(res))
#        return res
    
#class Fun(Fun):
#    def subst(self,v,a):
#        if self.var == v:
#            return self  # v is bound and not replaced
#        else: 
#            new_domain = substitute(self.domain_type,v,a) #extracted calls to substitute
#            new_body = substitute(self.body,v,a)
#            return Fun(self.var,new_domain,new_body)
#
        
def GeneralizedJoinType(Ts):
    if len(Ts)==0:
        return None
    elif len(Ts)==1:
        return Ts[0]
    else:
        return JoinType(Ts[0],GeneralizedJoinType(Ts[1:]))
        

# Very basic update

Robin's revised version of `IllocProp`.

In [74]:
Ind = BType('Ind')
Ind.judge('A')
Ind.judge('B')
greet = Pred('greet',[Ind,Ind])
countergreet = Pred('countergreet',[Ind,Ind])
illocrels = [greet,countergreet]

#IllocProp = GeneralizedJoinType([RecType({'x':Ind,'y':Ind,
#                    'sit-type':(Fun('v1',Ind,Fun('v2',Ind,SingletonType(Ty,
#                                                                      PType(r,['v1','v2'])))),
#                               ['x','y'])}) for r in illocrels])

IllocProp = JoinType(RecType({'x':Ind,'y':Ind,
                   'sit_type':(Fun('v1',Ind,Fun('v2',Ind,SingletonType(Ty,
                                                                     PType(greet,['v1','v2'])))),
                              ['x','y'])}),
                     RecType({'x':Ind,'y':Ind,
                   'sit_type':(Fun('v1',Ind,Fun('v2',Ind,SingletonType(Ty,
                                                                     PType(countergreet,['v1','v2'])))),
                              ['x','y'])}))

#IllocProp = RecType({'x':Ind,'y':Ind,
#                   'sit_type':(Fun('v1',Ind,Fun('v2',Ind,SingletonType(Ty,
#                                                                     PType(greet,['v1','v2'])))),
#                              ['x','y'])})

greetAB = PType(greet,['A','B'])

M = Possibility('M')
greetAB.in_poss(M).judge('e')

illocPropRec = Rec({'x':'A','y':'B','sit_type':greetAB})

print(IllocProp.query(illocPropRec))

#print(IllocProp.in_poss(M).query(illocPropRec))






True


Another attempt to do something similar.  This follows the spirit of what Jonathan wants if not the letter.  An IllocProp contains a situation and a situation type which involves an illocutionary predicate ("relation").

In [81]:
AusProp = RecType({'sit':Re,
                  'sit_type':RecTy})
IllocPType = Type('IllocPType')
IllocPType.witness_conditions = [lambda x: isinstance(x,PType) and x.comps.pred in illocrels]
IllocRecTy = Type('IllocRecTy')
IllocRecTy.witness_conditions = [lambda x: isinstance(x,RecType) and IllocPType.query(x.comps.c1) ]
IllocProp = RecType({'sit':Re,
                    'sit_type':IllocRecTy})

sit1 = Rec({'c1':'e'})
c1GreetAB = RecType({'c1':greetAB})
illoc1 = Rec({'sit':sit1,
             'sit_type':c1GreetAB})
IllocProp.query(illoc1)

True

In [79]:
T = RecType({'c1':Ind})
show(T.comps.c1)

'Ind'

It turns out that if you actually type in our original suggestion properly (we had a number of errors which I corrected) and you make the changes to SingletonType in the first cell, then it works.  As below:

In [2]:
Ind = BType('Ind')
Ind.judge('A')
Ind.judge('B')
greet = Pred('Greet',[Ind,Ind])
countergreet = Pred('CounterGreet',[Ind,Ind])

#Greet = lambda x, y : PType(greet, [x,y])
Greet = Fun('x', Ind, Fun('y', Ind, PType(greet, ['x','y'])))
#Greet = RecType({'x':Ind, 'y':Ind, 'e':(Fun('v1', Ind, Fun('v2', Ind, PType(greet, ['v1', 'v2']))), ['x','y'])})

print(show(Greet.appc('A').appc('B')))
GreetABRecType = RecType({'c1':Greet.app('A').app('B')})
print(show(GreetABRecType))

IllocRel = Type('IllocRel')
IllocRel.judge(Greet)

IllocProp = RecType({"R": IllocRel,
                               "x": Ind,
                               "y": Ind,
                               "sit-type": (Fun('v1',IllocRel,
                                     Fun('v2',Ind, 
                                         Fun('v3',Ind,
                                             SingletonType(RecTy, RecType({'c1': LazyObj([LazyObj(['v1','@','v2'])
                                                                     ,'@','v3'])}))))),
                                  ['R','x','y']
                                 )})
#IllocProp = RecType({"R": IllocRel,
#                    "x": Ind,
#                    "y": Ind,
#                    "illocptype":(Fun('v1',IllocRel,Fun('v2',Ind,Fun('v3',Ind, SingletonType(Ty,LazyObj([LazyObj(['v1','@','v2'])
#                                                                     ,'@','v3'])) ))),["R","x","y"])
#                    "sit-type": SingeltonType(RecTy,RecType({'c1'}))
#                                 )})

illocPropRec = Rec({"R": Greet, "x":"A", "y":"B", "sit-type": GreetABRecType})

#show(IllocProp.query(illocPropRec))
print(show(illocPropRec))
print(show(IllocProp))
IllocProp.query(illocPropRec)
#M = Possibility('M')
#show(Greet.appc_m('A',M).appc_m('B',M))
#show(IllocProp.in_poss(M).query(illocPropRec))

Greet(A, B)
{c1 : Greet(A, B)}
{R = lambda x:Ind . lambda y:Ind . Greet(x, y), x = A, y = B, sit-type = {c1 : Greet(A, B)}}
{R : IllocRel, x : Ind, y : Ind, sit-type : (lambda v1:IllocRel . lambda v2:Ind . lambda v3:Ind . RecTy_{c1 : [[v1, @, v2], @, v3]}, [R, x, y])}


True

In [ ]:


IllocProp = lambda r: RecType({"R": IllocRel,
                     "x": Ind,
                     "y": Ind,
                     "sit-type": (Fun('v1',Ind,
                                     Fun('v2',Ind, 
                                         PType(r, ['v1','v2']))))
                    })



In [327]:
show(Fun('v1',Ind,
    Fun('v2',Ind, 
        Fun('v3',IllocRel,
            RecType({'c1': LazyObj([LazyObj(['v3','@','v1'])
                                    ,'@','v2'])})))).app("A").app("B").app(Greet))

'{c1 : Greet(A, B)}'

In [328]:
IllocProp = RecType({"R": IllocRel,
                     "x": Ind,
                     "y": Ind,})
illocPropRec = Rec({"R": Greet, "x":"A", "y":"B"})
show(IllocProp.query(illocPropRec))

'True'

In [307]:
PreCond = RecType({'spkr':Ind,
                   'addr':Ind,
                   'moves': SingletonType(ListType(IllocProp), [])})

In [309]:
greeting = Fun('r', 
               PreCond, 
               RecType({'moves': SingletonType(ListType(IllocProp),
                                               [Greet.app(LazyObj(['r','.','spkr'])).app(LazyObj(['r','.','addr']))])}))
show(greeting)

'lambda r:{spkr : Ind, addr : Ind, moves : [{R : IllocRel}]_[]} . {moves : [{R : IllocRel}]_[Greet([r, ., spkr], [r, ., addr])]}'

In [281]:
greeting = Fun('r', 
               PreCond, 
               RecType({'moves': SingletonType(ListType(IllocProp),
                                               ListType(PType(greet,[LazyObj(['r','.','spkr']),
                                                                     LazyObj(['r','.','addr'])])))}))
show(greeting)

'lambda r:{spkr : Ind, addr : Ind, moves : [T8]_[]} . {moves : [T8]_[Greet([r, ., spkr], [r, ., addr])]}'

In [282]:
no_turn_change = Fun('r', 
                    RecType({'spkr': Ind, 'addr': Ind}),
                    RecType({'spkr': SingletonType(Ind, LazyObj(['r','.','spkr'])),
                             'addr': SingletonType(Ind, LazyObj(['r','.','addr']))}))
turn_change =    Fun('r', 
                    RecType({'spkr': Ind, 'addr': Ind}),
                    RecType({'spkr': SingletonType(Ind, LazyObj(['r','.','addr'])),
                             'addr': SingletonType(Ind, LazyObj(['r','.','spkr']))}))

In [284]:
Ind.judge('A')
Ind.judge('B')

#IS = Rec({'spkr':'A', 'addr':'B', 'moves':[]})
DGBType = RecType({'spkr':SingletonType(Ind, 'A'), 
                    'addr':SingletonType(Ind, 'B'), 
                    'moves': SingletonType(ListType(IllocProp), [])})
IS = DGBType.create()
print("initial IS     :", show(IS))
print("DGB type for it:", show(DGBType))

initial IS     : {spkr = A, addr = B, moves = []}
DGB type for it: {spkr : Ind_A, addr : Ind_B, moves : [T8]_[]}


In [277]:
print(show(DGBType), show(IS))
DGBType = DGBType.amerge(turn_change.appc(IS))
IS = DGBType.create()
print(show(DGBType), show(IS))
DGBType = DGBType.amerge(greeting.appc(IS))
IS = DGBType.create()
print(show(DGBType), show(IS))

{spkr : Ind_A, addr : Ind_B, moves : [T8]_[]} {spkr = A, addr = B, moves = []}
{spkr : Ind_B, addr : Ind_A, moves : [T8]_[]} {spkr = B, addr = A, moves = []}
{moves : [T8]_[Greet(B, A)], spkr : Ind_B, addr : Ind_A} {moves = [Greet(B, A)], spkr = B, addr = A}


'{spkr = _a46, addr = _a47, moves = [Greet(A, B)]}'